In [55]:
import numpy as np
import pandas as pd

In [56]:
# read the dataset
trainingdata = pd.read_csv('./dataset/banknote.csv')
labels = trainingdata.columns.values
print(labels)


['variance' 'skewness' 'curtosis' 'entropy' 'class']


In [57]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
# separate the column and the row
def scale_dataset(dataframe, oversample=False):
    X = dataframe[dataframe.columns[:-1]].values
    Y = dataframe[dataframe.columns[-1]].values

    scalar = StandardScaler()
    # take x and fit the scalar transform all the values
    X = scalar.fit_transform(X)
    # if one dataset is way much higher than the other one this will make the model to prefer the data with higher number of class
    # so we make the data equal
    if oversample:
        ros = RandomOverSampler()
        X, Y = ros.fit_resample(X, Y)

    return X, Y

In [58]:
from sklearn.model_selection import train_test_split
# scale and normalize the data
xtrain, ytrain = scale_dataset(trainingdata, oversample=True)
Xtrain,  Ytrain, Xtest, Ytest = train_test_split(xtrain, ytrain,  random_state=104,
                                                 test_size=0.25,
                                                 shuffle=True)



In [86]:
class RBFNN:

    def __init__(self, kernels, centers, beta=0.1, lr=0.1, epochs=80) -> None:

        self.kernels = kernels
        self.centers = centers
        self.beta = beta
        self.lr = lr
        self.epochs = epochs
        self.b = np.random.randn(1, 1)
        self.W = np.random.randn(kernels, 1)
       

      

        # to save the gradients calculated by the network
   
        self.gradients = []

    def rbf_activation(self, x, center):
        return np.exp(-self.beta*np.linalg.norm(x - center)**2)

    def linear_activation(self, A):
        return self.W.T.dot(A) + self.b

    def least_square_error(self, pred, y):
        return (y - pred).flatten()**2

    def _forward_propagation(self, x):

        a1 = np.array([
            [self.rbf_activation(x, center)]
            for center in self.centers
        ])

        a2 = self.linear_activation(a1)

        return a2, a1

    def _backpropagation(self, y, pred, a1):
        # Back propagation
        dW = -(y - pred).flatten()*a1
        db = -(y - pred).flatten()

        # Updating the weights
        self.W = self.W - self.lr*dW
        self.b = self.b - self.lr*db
        return dW, db

    def fit(self, X, Y):

        for _ in range(self.epochs):

            for x, y in list(zip(X, Y)):
                # Forward propagation
                pred, a1 = self._forward_propagation(x)

                error = self.least_square_error(pred[0], y[0, np.newaxis])
                

                # Back propagation
                dW, db = self._backpropagation(y, pred, a1)
                self.gradients.append((dW, db))

    def predict(self, x):
        a2, a1 = self._forward_propagation(x)
        
        return np.squeeze(a2) 

In [97]:
Xtest = Xtest.reshape(len(Xtest), 1)
Ytest = Ytest.reshape(len(Ytest), 1)

rbf = RBFNN(kernels=2,
                centers=np.array([
                    [0, 1 ,0 ,1],
                    [1, 0 ,1 , 0]

                ]),
                beta=0.31,
                lr=0.01,
                epochs=100
                )

rbf.fit(Xtrain, Xtest)


num_correct = 0

print("start predicting")

for i in range(len(Ytrain)):
    pred = rbf.predict(Ytrain[i])
    p = 1 if pred > 0.5 else 0
    
    if p == Ytest[i]:
         num_correct += 1
   
print(f' accuracy {(num_correct / len(Ytest)) * 100 } ')



start predicting
 accuracy 76.9028871391076 
